In [1]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints._base import Endpoint
from nba_api.stats.library.http import NBAStatsHTTP
from nba_api.stats.endpoints import playerdashboardbyteamperformance
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import teamdashboardbyteamperformance
from nba_api.stats.endpoints import playerdashboardbylastngames
from nba_api.stats.endpoints import playerdashboardbyyearoveryear
from nba_api.stats.endpoints import teamdashboardbylastngames
from nba_api.stats.endpoints import teamplayerdashboard
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import teams
from nba_api.stats.static import players
import pandas as pd
import inspect
import numpy as np
import csv
import sys

import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline

headers = { 'Host': 'stats.nba.com', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0', 'Accept': 'application/json, text/plain, /', 'Accept-Language': 'en-US,en;q=0.5', 'Referer': 'https://stats.nba.com/', 'Accept-Encoding': 'gzip, deflate, br', 'Connection': 'keep-alive', }

In [39]:
#Enter Team1

team1_input = input('Enter team 1: ')

print(team1_input)

team1 = teams.find_teams_by_full_name(team1_input)[0]['id']

team1_dashboard_adv = teamdashboardbylastngames.TeamDashboardByLastNGames(team_id=team1, season='2019-20', 
                                                                    measure_type_detailed_defense='Advanced', 
                                                                        last_n_games=5, headers=headers)

team1_headers = team1_dashboard_adv.overall_team_dashboard.get_dict().get('headers')
team1_data = team1_dashboard_adv.overall_team_dashboard.get_dict().get('data')[0]

print(team1, '\n', team1_headers[7], team1_data[7], '\n', 
      team1_headers[8], team1_data[8], '\n',
      team1_headers[9], team1_data[9], '\n', 
      team1_headers[10], team1_data[10])

E_OFF1_RATING = team1_data[7]
OFF1_RATING = team1_data[8]
E_DEF1_RATING = team1_data[9]
DEF1_RATING = team1_data[10]

Enter team 1: spurs
spurs
1610612759 
 E_OFF_RATING 103.1 
 OFF_RATING 103.3 
 E_DEF_RATING 105.3 
 DEF_RATING 107.2


In [40]:
#Enter Team2
team2_input = input('Enter team 2: ')
print(team2_input)

team2 = teams.find_teams_by_full_name(team2_input)[0]['id']

team2_dashboard_adv = teamdashboardbylastngames.TeamDashboardByLastNGames(team_id=team2, season='2019-20', 
                                                                    measure_type_detailed_defense='Advanced', 
                                                                        last_n_games=5, headers=headers)

team2_headers = team2_dashboard_adv.overall_team_dashboard.get_dict().get('headers')
team2_data = team2_dashboard_adv.overall_team_dashboard.get_dict().get('data')[0]

print(team2, '\n', team2_headers[7], team2_data[7], '\n', 
      team2_headers[8], team2_data[8], '\n',
      team2_headers[9], team2_data[9], '\n', 
      team2_headers[10], team2_data[10])

E_OFF2_RATING = team2_data[7]
OFF2_RATING = team2_data[8]
E_DEF2_RATING = team2_data[9]
DEF2_RATING = team2_data[10]

Enter team 2: grizzlies
grizzlies
1610612763 
 E_OFF_RATING 114.4 
 OFF_RATING 116.0 
 E_DEF_RATING 110.9 
 DEF_RATING 112.9


In [41]:
#Load Rosters
roster1 = teamplayerdashboard.TeamPlayerDashboard(team_id=team1, last_n_games=1, 
                                                  season='2019-20', headers=headers).get_dict()
roster2 = teamplayerdashboard.TeamPlayerDashboard(team_id=team2, last_n_games=1, 
                                                  season='2019-20', headers=headers).get_dict()

team1_roster_lastn = []
team1_roster_id = []
team2_roster_lastn = []
team2_roster_id = []

for n in roster1['resultSets'][1]['rowSet']:
    print(n[2], 'GP:', n[3], 'id:', n[1], 'Min', n[7])
    team1_roster_lastn.append(n[2])
    team1_roster_id.append(n[1])

for n in roster2['resultSets'][1]['rowSet']:
    print(n[2], 'GP:', n[3], 'id:', n[1], 'Min', n[7])
    team2_roster_lastn.append(n[2])
    team2_roster_id.append(n[1])
    
print(team1_roster_lastn, '\n',
     team2_roster_lastn)

Bryn Forbes GP: 1 id: 1627854 Min 19.466666666666665
Chimezie Metu GP: 1 id: 1629002 Min 8.25
DeMar DeRozan GP: 1 id: 201942 Min 24.77
DeMarre Carroll GP: 1 id: 201960 Min 7.666666666666667
Dejounte Murray GP: 1 id: 1627749 Min 23.533333333333335
Derrick White GP: 1 id: 1628401 Min 20.716666666666665
Jakob Poeltl GP: 1 id: 1627751 Min 15.733333333333333
LaMarcus Aldridge GP: 1 id: 200746 Min 24.016666666666666
Lonnie Walker IV GP: 1 id: 1629022 Min 16.95
Marco Belinelli GP: 1 id: 201158 Min 25.063333333333333
Patty Mills GP: 1 id: 201988 Min 20.033333333333335
Rudy Gay GP: 1 id: 200752 Min 20.233333333333334
Trey Lyles GP: 1 id: 1626168 Min 13.566666666666666
Brandon Clarke GP: 1 id: 1629634 Min 18.75
Bruno Caboclo GP: 1 id: 203998 Min 4.8
De'Anthony Melton GP: 1 id: 1629001 Min 14.933333333333334
Dillon Brooks GP: 1 id: 1628415 Min 29.866666666666667
Grayson Allen GP: 1 id: 1628960 Min 20.55
Ja Morant GP: 1 id: 1629630 Min 30.15
Jae Crowder GP: 1 id: 203109 Min 30.65
Jaren Jackson Jr.

In [42]:
#Individual Off and Def Ratings
    
team1_players_off = []
team1_players_def = []
team2_players_off = []
team2_players_def = []

roster1_calc = []
roster2_calc = []
roster1_calc_id = []
roster2_calc_id = []
roster1_pos = []
roster2_pos = []

player_info_off = leaguedashplayerstats.LeagueDashPlayerStats(last_n_games=5, season='2019-20', 
                                                              measure_type_detailed_defense='Advanced')

player_off_headers = player_info_off.league_dash_player_stats.get_dict().get('headers')
player_off_data = player_info_off.league_dash_player_stats.get_dict().get('data')

for p in team1_roster_lastn:
    for n in player_off_data:
        if p in n:
            print(n[1], '\n', 'Min', n[9], 
                  'E_Off_Rating', n[10], 'E_Def_Rating', n[13])
            if n[9] > 10:
                roster1_calc.append(n[1])
                team1_players_off.append(n[10])
                team1_players_def.append(n[13])
                roster1_calc_id.append(n[0])

for p in team2_roster_lastn:
    for n in player_off_data:
        if p in n:
            print(n[1], '\n', 'Min', n[9], 
                  'E_Off_Rating', n[10], 'E_Def_Rating', n[13])
            if n[9] > 10:
                roster2_calc.append(n[1])
                team2_players_off.append(n[10])
                team2_players_def.append(n[13])
                roster2_calc_id.append(n[0])
print(roster2_calc)

# #Player Position

# for pid in roster1_calc_id:
#     player_pos_info = commonplayerinfo.CommonPlayerInfo(player_id=pid, headers=headers)
#     player_data_calc = player_pos_info.common_player_info.get_dict().get('data')[0]
#     print(player_data_calc[3], player_data_calc[0], player_data_calc[14])
#     roster1_pos.append(player_data_calc[14])

# for pid in roster2_calc_id:
#     player_pos_info = commonplayerinfo.CommonPlayerInfo(player_id=pid, headers=headers)
#     player_data_calc = player_pos_info.common_player_info.get_dict().get('data')[0]    
#     print(player_data_calc[3], player_data_calc[0], player_data_calc[14])
#     roster2_pos.append(player_data_calc[14])

# data1 = {'Name':roster1_calc, 'OffR':team1_players_off, 'DefR':team1_players_def, 'Pos': roster1_pos}
# data2 = {'Name':roster2_calc, 'OffR':team2_players_off, 'DefR':team2_players_def, 'Pos': roster2_pos}

Bryn Forbes 
 Min 20.5 E_Off_Rating 97.2 E_Def_Rating 106.4
Chimezie Metu 
 Min 4.5 E_Off_Rating 94.3 E_Def_Rating 83.7
DeMar DeRozan 
 Min 32.2 E_Off_Rating 103.7 E_Def_Rating 116.2
DeMarre Carroll 
 Min 7.7 E_Off_Rating 99.5 E_Def_Rating 86.6
Dejounte Murray 
 Min 25.4 E_Off_Rating 102.2 E_Def_Rating 116.5
Derrick White 
 Min 23.8 E_Off_Rating 104.2 E_Def_Rating 91.6
Jakob Poeltl 
 Min 16.0 E_Off_Rating 114.3 E_Def_Rating 91.6
LaMarcus Aldridge 
 Min 34.0 E_Off_Rating 97.6 E_Def_Rating 112.0
Lonnie Walker IV 
 Min 17.3 E_Off_Rating 103.7 E_Def_Rating 86.9
Marco Belinelli 
 Min 17.8 E_Off_Rating 112.2 E_Def_Rating 114.0
Patty Mills 
 Min 25.0 E_Off_Rating 104.0 E_Def_Rating 107.0
Rudy Gay 
 Min 21.6 E_Off_Rating 101.2 E_Def_Rating 90.3
Trey Lyles 
 Min 13.1 E_Off_Rating 100.8 E_Def_Rating 119.2
Brandon Clarke 
 Min 22.2 E_Off_Rating 111.1 E_Def_Rating 106.7
Bruno Caboclo 
 Min 4.7 E_Off_Rating 128.6 E_Def_Rating 119.0
De'Anthony Melton 
 Min 19.0 E_Off_Rating 115.4 E_Def_Rating 102.0


In [43]:
#Theory: If second game of back-to-back is a road game, odds of winning decrease 7-8%, multiply ratings by 0.93.

In [44]:
#Individual Off_PER 

def off_per_calculator(player, player_id, team):
    
    PER_MP = 0
    agg_aPERmin = 0
    agg_min = 0
    p_name = []
    
    teamdashboard = teamdashboardbylastngames.TeamDashboardByLastNGames(team_id=team, season='2019-20', 
                                                                        last_n_games=5, headers=headers)
    teamdashboard_adv = teamdashboardbylastngames.TeamDashboardByLastNGames(team_id=team, season='2019-20', 
                                                                    measure_type_detailed_defense='Advanced', 
                                                                        last_n_games=5, headers=headers)

    ##last n games
    team_headers_basic = teamdashboard.overall_team_dashboard.get_dict().get('headers')
    team_data_basic = teamdashboard.overall_team_dashboard.get_dict().get('data')[0]
    team_headers = teamdashboard_adv.overall_team_dashboard.get_dict().get('headers')
    team_data = teamdashboard_adv.overall_team_dashboard.get_dict().get('data')[0]
    
    for n in player_id:    
        p_info = playerdashboardbylastngames.PlayerDashboardByLastNGames(player_id=n, season='2019-20',
                                                                        last_n_games=5, headers=headers)
        if players.find_player_by_id(n):
            p_name = players.find_player_by_id(player_id=n)['full_name']
        player_headers = p_info.overall_player_dashboard.get_dict().get('headers')
        player_data = p_info.overall_player_dashboard.get_dict().get('data')[0]
 
    #####PER player stats####
        MP = player_data[6]
        ThrP = player_data[10] 
        AST = player_data[19]
        TOV = player_data[20]
        FGA = player_data[8]
        FG = player_data[7] + player_data[10] 
        FTA = player_data[14]
        FT = player_data[13]
        TRB = player_data[18]
        ORB = player_data[16]
        STL = player_data[21]
        BLK = player_data[22]
        PF = player_data[24]

    ###PER league stats (2018-19)###
        lg_AST = 24.6
        lg_FG = 41.1
        lg_PTS = 111.2
        lg_FGA = 89.2
        lg_FT = 17.7
        lg_FTA = 23.1
        lg_TRB = 45.2
        lg_ORB = 10.3
        lg_TOV = 14.1
        lg_PF = 20.9
        lg_pace = 100
        
    ###PER team stats###
        team_AST = team_data_basic[19]
        team_FG = team_data_basic[7]+team_data_basic[10]
        team_pace = team_data[23] 
       
        factor = (2/3) - (0.5*(lg_AST/lg_FG)) / (2*(lg_FG/lg_FT)) 
        VOP = lg_PTS / (lg_FGA-lg_ORB+lg_TOV+0.44*lg_FTA) 
        DRB_perc = (lg_TRB -lg_ORB) / lg_TRB

    ####PER Calculation####                    
        PER_calc_player = (1 / MP) * ( ThrP + (2/3) * AST 
                                      + (2 - factor * (team_AST / team_FG)) * FG 
                                      + (FT *0.5 * (1 + (1 - (team_AST / team_FG)) + (2/3) * (team_AST / team_FG)))
                                      - VOP * TOV - VOP * DRB_perc * (FGA - FG)
                                      - VOP * 0.44 * (0.44 + (0.56 * DRB_perc)) * (FTA - FT)
                                      + VOP * (1 - DRB_perc) * (TRB - ORB) + VOP * DRB_perc * ORB 
                                      + VOP * STL + VOP * DRB_perc * BLK - PF 
                                      * ((lg_FT / lg_PF) - 0.44 * (lg_FTA / lg_PF) * VOP) )

        adj_PER_calc_player = PER_calc_player * (lg_pace /team_pace) *54
        aPER_min = adj_PER_calc_player * MP
        agg_aPERmin += aPER_min
        agg_min += MP
        
        print(p_name, n, 'Min', round(MP, 2), 
              'adj_PER_calc_player', round(adj_PER_calc_player, 2), 
              'aPER_min', round(aPER_min, 2))

    print('agg_aPERmin', round(agg_aPERmin,2), 
          'avg_aPER_min', round(agg_aPERmin/agg_min, 2), '\n')

In [45]:
off_per_calculator(team1_roster_lastn, team1_roster_id, team1)
off_per_calculator(team2_roster_lastn, team2_roster_id, team2)

Bryn Forbes 1627854 Min 102.51 adj_PER_calc_player 10.86 aPER_min 1113.65
Chimezie Metu 1629002 Min 9.05 adj_PER_calc_player 5.07 aPER_min 45.85
DeMar DeRozan 201942 Min 160.79 adj_PER_calc_player 14.77 aPER_min 2374.08
DeMarre Carroll 201960 Min 7.67 adj_PER_calc_player -0.41 aPER_min -3.18
Dejounte Murray 1627749 Min 127.12 adj_PER_calc_player 29.42 aPER_min 3739.24
Derrick White 1628401 Min 118.84 adj_PER_calc_player 25.56 aPER_min 3037.17
Jakob Poeltl 1627751 Min 79.85 adj_PER_calc_player 24.5 aPER_min 1956.11
LaMarcus Aldridge 200746 Min 169.89 adj_PER_calc_player 22.6 aPER_min 3839.56
Lonnie Walker IV 1629022 Min 86.54 adj_PER_calc_player 13.84 aPER_min 1197.78
Marco Belinelli 201158 Min 88.99 adj_PER_calc_player 24.89 aPER_min 2215.39
Patty Mills 201988 Min 125.14 adj_PER_calc_player 37.0 aPER_min 4630.61
Rudy Gay 200752 Min 108.12 adj_PER_calc_player 16.88 aPER_min 1824.91
Trey Lyles 1626168 Min 65.5 adj_PER_calc_player 13.58 aPER_min 889.52
agg_aPERmin 26860.7 avg_aPER_min 21.

In [46]:
#Alt Player Def Rating

def def_per_calc(team_roster):
    agg_def_rating = 0
    agg_min = 0
    player_info_def = leaguedashplayerstats.LeagueDashPlayerStats(last_n_games=5, season='2019-20', 
                                                                      measure_type_detailed_defense='Defense')

    player_def_headers = player_info_def.league_dash_player_stats.get_dict().get('headers')
    player_def_data = player_info_def.league_dash_player_stats.get_dict().get('data')

#     print('headers:', player_def_headers)
        #print('data:', player_def_data)

    for p in team_roster:
        for n in player_def_data:
            if p in n:
                print(n[1], '\n', 'Def Rating', n[10], 'Min', n[9])
                agg_def_rating += (n[10]*n[9])
                agg_min += n[9]

    
    avg_def_rating = agg_def_rating/len(team_roster)
    avg_def_rating_min = agg_def_rating/agg_min

    print('avg_def_rating', avg_def_rating, 'agg_min', round(agg_min, 2)) 
    print('avg_def_rating_min', round(avg_def_rating_min, 2), '\n')

In [47]:
def_per_calc(team1_roster_lastn)
def_per_calc(team2_roster_lastn)

Bryn Forbes 
 Def Rating 108.2 Min 102.51333333333334
Chimezie Metu 
 Def Rating 85.0 Min 9.046666666666667
DeMar DeRozan 
 Def Rating 118.6 Min 160.79
DeMarre Carroll 
 Def Rating 88.2 Min 7.666666666666667
Dejounte Murray 
 Def Rating 118.4 Min 127.12
Derrick White 
 Def Rating 92.0 Min 118.84333333333333
Jakob Poeltl 
 Def Rating 90.8 Min 79.84833333333333
LaMarcus Aldridge 
 Def Rating 113.1 Min 169.89
Lonnie Walker IV 
 Def Rating 86.5 Min 86.54
Marco Belinelli 
 Def Rating 113.4 Min 88.99166666666666
Patty Mills 
 Def Rating 107.4 Min 125.13666666666667
Rudy Gay 
 Def Rating 90.8 Min 108.11666666666666
Trey Lyles 
 Def Rating 118.6 Min 65.49666666666667
avg_def_rating 10204.471282051281 agg_min 1250.0
avg_def_rating_min 106.13 

Brandon Clarke 
 Def Rating 107.4 Min 110.96333333333334
Bruno Caboclo 
 Def Rating 110.5 Min 9.366666666666667
De'Anthony Melton 
 Def Rating 101.0 Min 94.96666666666667
Dillon Brooks 
 Def Rating 116.6 Min 145.1
Grayson Allen 
 Def Rating 96.9 Min 31.23

#########

In [ ]:
#Player Position

for pid in roster1_calc_id:
    player_pos_info = commonplayerinfo.CommonPlayerInfo(player_id=pid, headers=headers)
    player_data_calc = player_pos_info.common_player_info.get_dict().get('data')[0]
    print(player_data_calc[3], player_data_calc[0], player_data_calc[14])
    roster1_pos.append(player_data_calc[14])

for pid in roster2_calc_id:
    player_pos_info = commonplayerinfo.CommonPlayerInfo(player_id=pid, headers=headers)
    player_data_calc = player_pos_info.common_player_info.get_dict().get('data')[0]    
    print(player_data_calc[3], player_data_calc[0], player_data_calc[14])
    roster2_pos.append(player_data_calc[14])

data1 = {'Name':roster1_calc, 'OffR':team1_players_off, 'DefR':team1_players_def, 'Pos': roster1_pos}
data2 = {'Name':roster2_calc, 'OffR':team2_players_off, 'DefR':team2_players_def, 'Pos': roster2_pos}

In [53]:
pdf1 = pd.DataFrame(data1, columns = ['Name', 'OffR', 'DefR', 'Pos'])  
pdf2 = pd.DataFrame(data2, columns = ['Name', 'OffR', 'DefR', 'Pos'])

In [54]:
from pandas import DataFrame
from sklearn import linear_model
import statsmodels.api as sm
# prediction with sklearn
#https://stats.nba.com/team/1610612763/advanced/?Season=2019-20&SeasonType=Regular%20Season&Split=lastn

New_T1_OR5 = E_OFF1_RATING
New_T1_DR5 = E_DEF1_RATING
New_T1_Diff = (E_OFF1_RATING - E_DEF1_RATING)

New_T2_OR5 = E_OFF2_RATING
New_T2_DR5 = E_DEF2_RATING
New_T2_Diff = (E_OFF2_RATING - E_DEF2_RATING)

print('Predicted Spread: \n', regr.predict([[New_T1_OR5 ,New_T1_DR5, 
                                             New_T2_OR5, New_T2_OR5]]))
print('Predicted Over/Under: \n', regr2.predict([[New_T1_OR5 ,New_T1_DR5,
                                                  New_T2_OR5, New_T2_OR5 ]]))

# print(New_T1_Diff, New_T2_Diff)
# print('Predicted Spread: \n', regr.predict([[New_T1_Diff, New_T2_Diff]]))
# print('Predicted Over/Under: \n', regr2.predict([[New_T1_Diff, New_T2_Diff]]))

# with statsmodels
X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

Predicted Spread: 
 [-2.03493686]
Predicted Over/Under: 
 [219.46703766]
                            OLS Regression Results                            
Dep. Variable:                 Spread   R-squared:                       0.101
Model:                            OLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     64.67
Date:                Mon, 09 Dec 2019   Prob (F-statistic):           6.76e-52
Time:                        19:59:54   Log-Likelihood:                -9365.1
No. Observations:                2310   AIC:                         1.874e+04
Df Residuals:                    2305   BIC:                         1.877e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [55]:
# prediction with sklearn
from pandas import DataFrame
from sklearn import linear_model
import statsmodels.api as sm

tpt_total = 0

for n in range(len(data1['Name'])):
    ppt_total = 0
    count = 0
    print('=== \n', data1['Name'][n], data1['OffR'][n], data1['DefR'][n], data1['Pos'][n])
    for p in range(len(data2['Name'])):
        if data1['Pos'][n] == data2['Pos'][p]:
            print(data2['Name'][p])
            print('d1or', data1['OffR'][n], 'd2dr', data2['DefR'][p])
            New_T1_OR5 = data1['OffR'][n]
            New_T1_DR5 = data1['DefR'][n]
            New_T2_OR5 = data2['OffR'][p]
            New_T2_DR5 = data2['DefR'][p]

            print('Predicted Spread: \n', regr.predict([[New_T1_OR5 ,New_T1_DR5, New_T2_OR5, New_T2_OR5 ]]))
    #             print('Predicted Over/Under: \n', regr2.predict([[New_T1_OR5 ,New_T1_DR5, New_T2_OR5, New_T2_OR5 ]])) 
            spread = regr.predict([[New_T1_OR5 ,New_T1_DR5, New_T2_OR5, New_T2_OR5 ]])
            ppt_total += spread
            tpt_total += spread
            
    print('player_pt_total', ppt_total, '\n')

print('team_pt_total', tpt_total)
            
#     # with statsmodels
#     X = sm.add_constant(X) # adding a constant

#     model = sm.OLS(Y, X).fit()
#     predictions = model.predict(X) 

#     print_model = model.summary()
#     print(print_model)

=== 
 Alex Len 106.5 102.0 Center
player_pt_total 0 

=== 
 Allen Crabbe 114.2 129.1 Guard-Forward
player_pt_total 0 

=== 
 Cam Reddish 104.4 100.8 Forward-Guard
player_pt_total 0 

=== 
 Damian Jones 112.3 111.2 Center
player_pt_total 0 

=== 
 De'Andre Hunter 109.7 113.0 Forward-Guard
player_pt_total 0 

=== 
 DeAndre' Bembry 102.8 111.5 Guard-Forward
player_pt_total 0 

=== 
 Jabari Parker 110.2 120.0 Forward
Derrick Jones Jr.
d1or 110.2 d2dr 98.8
Predicted Spread: 
 [-3.79419268]
Duncan Robinson
d1or 110.2 d2dr 99.1
Predicted Spread: 
 [-3.91582498]
Jimmy Butler
d1or 110.2 d2dr 105.7
Predicted Spread: 
 [-3.85500883]
player_pt_total [-11.5650265] 

=== 
 Kevin Huerter 114.5 95.2 Guard-Forward
player_pt_total 0 

=== 
 Trae Young 114.5 115.0 Guard
Kendrick Nunn
d1or 114.5 d2dr 104.2
Predicted Spread: 
 [0.11429342]
Tyler Herro
d1or 114.5 d2dr 113.1
Predicted Spread: 
 [0.06868131]
player_pt_total [0.18297473] 

=== 
 Vince Carter 97.2 102.8 Guard-Forward
player_pt_total 0 

team_pt

In [39]:
rdf = pd.read_csv("regression_list.csv", sep=r'\t', engine='python') 
rdf.T1_OR5

0       110.22
1       110.00
2       112.46
3       110.68
4       111.00
5       108.94
6       106.50
7       106.36
8       109.04
9       111.58
10      111.34
11      114.18
12      113.92
13      111.44
14      109.02
15      111.64
16      114.06
17      114.32
18      114.58
19      111.14
20      109.04
21      108.66
22      109.80
23      106.50
24      114.34
25      116.86
26      116.56
27      113.68
28      117.66
29      112.82
         ...  
2280    108.04
2281    114.84
2282    113.12
2283    110.22
2284    111.96
2285    112.08
2286    110.44
2287    109.16
2288    107.68
2289    104.60
2290    106.66
2291    106.76
2292    106.90
2293    112.16
2294    113.10
2295    108.26
2296    108.60
2297    111.54
2298    110.70
2299    111.52
2300    111.12
2301    109.62
2302    106.64
2303    103.90
2304    101.60
2305    102.16
2306    102.76
2307    102.62
2308    105.00
2309    107.06
Name: T1_OR5, Length: 2310, dtype: float64

In [ ]:
rdf = pd.read_csv("regression_list.csv", sep=r'\t', engine='python') 

rdf['T1_Diff'] = rdf.T1_OR5 - rdf.T1_DR5
rdf['T2_Diff'] = rdf.T2_OR5 - rdf.T2_DR5

rdf.T1_OR5 = pd.to_numeric(rdf.T1_OR5)
rdf.T1_DR5 = pd.to_numeric(rdf.T1_DR5)
rdf.T2_OR5 = pd.to_numeric(rdf.T2_OR5)
rdf.T2_DR5 = pd.to_numeric(rdf.T2_DR5)

rdf.plot(x='T1_Diff', y='Spread', style='o')  
rdf.plot(x='T2_Diff', y='Spread', style='o') 
rdf.plot(x='T1_Diff', y='TScore', style='o') 


In [40]:
from pandas import DataFrame
from sklearn import linear_model
import statsmodels.api as sm

X = rdf[['T1_OR5','T1_DR5', 'T2_OR5', 'T2_DR5']]
# X = rdf[['T1_Diff', 'T2_Diff']]

Y = rdf['Spread']
Y2 = rdf['TScore']

regr = linear_model.LinearRegression()
regr.fit(X, Y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

regr2 = linear_model.LinearRegression()
regr2.fit(X, Y2)

print('Intecept_Score: \n', regr2.intercept_)
print('Coefficients_Score: \n', regr2.coef_)


Intercept: 
 0.1968372047350489
Coefficients: 
 [ 0.42950031 -0.40077189 -0.43303045  0.40262237]
Intecept_Score: 
 96.94754180172698
Coefficients_Score: 
 [0.42136938 0.15357508 0.41373199 0.15386099]


/Users/stan/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [ ]:
data = pd.read_csv("NBA Merged Sheets - Sheet2.csv") 
data_t = pd.read_csv("team_stats_basic_2018-19.csv")
df = pd.read_csv("team 2018-19.csv") 

teams = ['ATL', 'IND', 'MIL', 'ORL', 'PHI', 'SAS', 'POR', 'NOP', 'UTA', 'HOU', 'BOS', 
         'MEM', 'BKN', 'MIA', 'CHI', 'MIN', 'PHX', 'DET', 'NYK', 'LAL', 'CHA', 'TOR', 
         'WAS', 'SAC', 'LAC', 'OKC', 'CLE', 'DAL', 'DEN', 'GSW']

In [22]:
def regression(team):
    
    with open('regression_list.csv', 'a', newline='') as csvFile:
        writer = csv.writer(csvFile)
        headers = ['Team1','T1_OR5', 'T1_DR5', 'Team2', 'T2_OR5', 'T2_DR5', 'Spread', 'TScore']
        writer.writerow(headers)
    csvFile.close() 
    
    team2 = []
    date = []
    score = []
    opp = []
    
    df2 = df[['Team', 'Matchup', 'Date', 'OffRtg', 'DefRtg', 'ASTRatio', 'PACE']]
    df2 = df[df['Team'] == team1]

    team2 = df2['Matchup']

    for row in team2:
        opp.append(row.replace(team1+'vs.', '').replace(team1+'@', ''))

    df2['Opp'] = opp

    team1_off_avg = 0
    team1_def_avg = 0
    team2_off_avg = 0
    team2_def_avg = 0
    totalscore = 0

    for index, row in df2.head(n=77).iterrows(): 
        whole_row = []
        team2 = row['Opp']
        date = row['Date']
        df3 = df[df['Team'] == team2]

        score = data.loc[(data['DATE']==date) & (data['TEAM']==team1)]
        score = score.iloc[0]
        score = (score["team_pts"], score['team_plus_minus'])
        print('===', row['Team'], row['OffRtg'], row['DefRtg'], 
              row['Date'], row['Opp'], score, "===") 
                
        print(row['Team'], 'Last5_Off', row['Last5_Off'], 'Last5_Def', row['Last5_Def'])
        whole_row.append(row['Team'])
        whole_row.append(row['Last5_Off'])
        whole_row.append(row['Last5_Def'])

        df3 = df3.loc[df3['Date'] == date]
        for index, row in df3.iterrows():
            print(row['Team'], 'Last5_Off', row['Last5_Off'], 'Last5_Def', row['Last5_Def'], '\n')
            whole_row.append(row['Team'])
            whole_row.append(row['Last5_Off'])
            whole_row.append(row['Last5_Def'])
            whole_row.append(score[1])
            whole_row.append(score[0]-score[1]+score[0])
            
            with open('regression_list.csv', 'a') as csvFile:
                writer = csv.writer(csvFile)
                writer.writerow(whole_row)
            csvFile.close() 

In [23]:
for team in teams:
    regression(team)

TypeError: 'module' object is not iterable